In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

In [2]:
#CHANGE LAST ELEMENT???
elements = pmg.Element('Li'), pmg.Element('Ni'), pmg.Element('O'), pmg.Element('Ge') #Ge is oxygen shell
li, ni, o, ge = elements

pairs = [(li,ge), (ni,ge), (ge,ge)]

charges = {li: 1, ni: 3, o: 0.960, ge: -2.960}
masses = {li: 6.9410, ni: 52.8237, o: 14.3991, ge: 1.5999}

bpp_def = {'Li_Ge_A'   : 456.783,
           'Li_Ge_rho' : 0.212,
           'Li_Ge_C'   : 0.0,
           'Ni_Ge_A'   : 1540.922,
           'Ni_Ge_rho' : 0.323,
           'Ni_Ge_C'   : 0.425,
           'Ge_Ge_A'   : 22764.7000,
           'Ge_Ge_rho' : 0.1490,
           'Ge_Ge_C'   : 20.4}

bpp = {'Li_Ge_A'   : 456.783,
       'Li_Ge_rho' : 0.212,
       'Li_Ge_C'   : 0.0,
       'Ni_Ge_A'   : 1540.922,
       'Ni_Ge_rho' : 0.323,
       'Ni_Ge_C'   : 0.425,
       'Ge_Ge_A'   : 22764.7000,
       'Ge_Ge_rho' : 0.1490,
       'Ge_Ge_C'   : 20.4}


In [3]:
structures = []
for structure in glob.glob('poscars/POSCAR*'):
    st = pmg.Structure.from_file(structure)
    structures.append(st)

In [4]:
for structure in structures:
    new_sites = []
    [new_sites.append(structure.frac_coords[i]) for i, species in enumerate(structure.species) if species.symbol == "O" ]
    for i, species in enumerate(new_sites):
        structure.append('Ge', new_sites[i], validate_proximity=False, properties=None)

In [5]:
def init(structure):
    lmp = lammps.Lammps(units='metal',
                        style = 'full',
                        args=['-log', 'none', '-screen', 'none'])
    lmp.system.add_pymatgen_structure(structure, elements)
    

    lmp.command('group rigid type 1 2')
    lmp.command('group coreshell type 3 4')
    lmp.command('group cores type 3')
    lmp.command('group shells type 4')

    lmp.command('pair_style buck/coul/long 12.0')
    lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('bond_style harmonic')
    lmp.command('bond_coeff 1 65.0 0.0')
    
    lmp.command('kspace_style pppm 1e-6')
    

    for c in charges:
        lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))
#     for i, m in enumerate(masses):
#         lmp.command('set atom %d mass %f' % (elements.index(m)+1, masses[m]))

    lmp.command('fix freeze cores setforce 0.0 0.0 0.0')

    return lmp

In [6]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
    
def set_potentials(instance):
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))



def dex(elin): return elements.index(elin) + 1

In [7]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([instances[0].system.total,3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.run(0)
            out[:,:,instances.index(instance)] = instance.system.forces
            
    else: out = np.ones([instances[0].system.total,3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
        
    return out

In [8]:
instances = [init(structure) for structure in structures]

LammpsNormalError: ERROR: Numeric index is out of bounds (../bond_harmonic.cpp:128)


In [ ]:
expected = np.zeros([instances[0].system.total,3, len(instances)])

In [ ]:
with pm.Model() as model:
    Li_Ge_A      = pm.Normal("Li_Ge_A",    mu = bpp_def['Li_Ge_A'],    sd =  100)
    Li_Ge_rho    = pm.Normal("Li_Ge_rho",  mu = bpp_def['Li_Ge_rho'],  sd =  0.05)
#     Li_Ge_C      = pm.Normal("Li_Ge_C",    mu = bpp_def['Li_Ge_C'],    sd =  0.01)
    Ni_Ge_A      = pm.Normal("Ni_Ge_A",    mu = bpp_def['Ni_Ge_A'],    sd =  100)
    Ni_Ge_rho    = pm.Normal("Ni_Ge_rho",  mu = bpp_def['Ni_Ge_rho'],  sd =  0.05)
    Ni_Ge_C      = pm.Normal("Ni_Ge_C",    mu = bpp_def['Ni_Ge_C'],    sd =  0.05)
    Ge_Ge_A       = pm.Normal("Ge_Ge_A",     mu = bpp_def['Ge_Ge_A'],     sd =  100)
    Ge_Ge_rho     = pm.Normal("Ge_Ge_rho",   mu = bpp_def['Ge_Ge_rho'],   sd =  0.05)
    Ge_Ge_C       = pm.Normal("Ge_Ge_C",     mu = bpp_def['Ge_Ge_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1), draws=500)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')

In [11]:
    lmp = lammps.Lammps(units='metal',
                        style = 'full',
                        args=['-log', 'none', '-screen', 'none'])
    lmp.system.add_pymatgen_structure(structures[0], elements)

    lmp.command('group rigid type 1 2')
    lmp.command('group coreshell type 3 4')
    lmp.command('group cores type 3')
    lmp.command('group shells type 4')

'group'